# 0. Import dependencies

In [ ]:
import torch
import sklearn
import numpy as np


In [ ]:
from sklearn.datasets import make_classification
torch.manual_seed(123)


# 1. Working with datasets

## 1.1 Creating the datasets
Creating a toy dataset with `sklearn.make_classification()`.

In [ ]:
# Make a Toy Dataset
X, y = make_classification(n_samples=1000, n_features=4, n_classes=2)

# Change it into the Tensor
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)


In [ ]:
print(f'X: {X.ndim} dim(s)\n', 'Shape of X: ', np.shape(X), '\n', X, sep='')

In [ ]:
print(f'y: {y.ndim} dim(s)\n', 'Shape of y: ', np.shape(y), '\n', y, sep='')

## 1.2 Shuffling the datasets
Shuffling using the `torch.randperm()` function. Learn more about this function [here](https://pytorch.org/docs/stable/generated/torch.randperm.html).

Here, the first argument needs to be the size `n` in `int`. The `y.size()` simply won't work since the `torch.Tensor.size()` function returns the size as an object. We pass in the first dimension (0th) to retrieve size of the first dim in `int`. Values other than 0 won't work since this tensor only has one dim. 

In [ ]:
# Shuffling data
torch.manual_seed(10)

# Creating a shuffled dataset ranging from 0 to n - 1 (999)
shuffledData = torch.randperm(y.size(0), dtype=torch.long)

# Printing a sample from our shuffled data
print(f'Sampled shuffled data:\n\n{shuffledData[:10]}')


In [ ]:
print(f'Shuffled generated data:\n\n{shuffledData}')

We've created a list of indices that are sorted. Now mapping the shuffled indices to create a shuffled dataset.

In [ ]:
X, y = X[shuffledData], y[shuffledData]

In [ ]:
print(f'Shuffled X:\n\n{X}')

In [ ]:
print(f'Shuffled y:\n\n{y}')

## 1.3 Splitting the dataset into training and testing data

### 1.3.1 Shuffling data

In [ ]:
# Train Test split
shuffledTrainDataMakeup = int(shuffledData.size(0) * 0.7)

shuffledTrainData = shuffledData[:shuffledTrainDataMakeup]
shuffledTestData = shuffledData[shuffledTrainDataMakeup:]


How much does the training data make up of the actual dataset.

In [ ]:
print(f'Training data makeup: {shuffledTrainDataMakeup}\n\n')


Details of the training data.

In [ ]:
print(f'Printing details of training data\n\nShape: {np.shape(shuffledTrainData)}\n\nData:\n{shuffledTrainData}')


Details of the testing data.

In [ ]:
print(f'Printing details of testing data\n\nShape: {np.shape(shuffledTestData)}\n\nData:\n{shuffledTestData}')


In [ ]:
X_train, y_train = X[shuffledTrainData], y[shuffledTrainData]
X_test, y_test = X[shuffledTestData], y[shuffledTestData]


Printing details of shuffled training and testing data

In [ ]:
X_train.shape


In [ ]:
X_train


In [ ]:
X_test.shape


In [ ]:
X_test


## 1.4 Normalising data

In [ ]:
mean, std = X_train.mean(dim=0), X_train.std(dim=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std


---

# 2. Building a Neural Network

Importing libraries

In [ ]:
import torch.nn as nn
from torch.autograd import grad


Class definition for the neural network

In [ ]:
class NN():
    def __init__(self, n_features):
        self.n_features = n_features
        self.weight = torch.zeros(
            size=(n_features, 1), dtype=torch.float, requires_grad=True)
        self.bias = torch.zeros(1, dtype=torch.float, requires_grad=True)

    def forward(self, x):
        output = torch.add(torch.mm(x, self.weight), self.bias)
        return output.view(-1)


Defining our loss function. Looks oddly similar to the mean square error function.

In [ ]:
def loss_fn(yHat, y):
    return torch.mean((yHat - y) ** 2)

Function to train the model returning the cost as a list.

In [ ]:
def train(model, x, y, n_epoch=10, lr=0.001, seed=23, bsz=50):
    cost = []
    torch.manual_seed(seed)
    
    for i in range(n_epoch):
        shuffledData = torch.randperm(y.size(0))
        batches = torch.split(shuffledData, bsz)

        for idx in batches:
            # 1. Compute the output
            yHat = model.forward(x[idx])

            # 2. Compute the Error
            loss = loss_fn(yHat, y[idx])

            # 3. Compute the gradient
            grad_w = grad(loss, model.weight, retain_graph=True)[0]
            grad_b = grad(loss, model.bias)[0]

            # 4. Update Model Parameters
            model.weight = model.weight - lr * grad_w
            model.bias = model.bias - lr * grad_b

        # 5. Log and print the loss
        with torch.no_grad():
            yHat = model.forward(x)
            curr_loss = loss_fn(yHat, y)
            
            print('Epoch: %3d ' % (i + 1), end="")
            print('| MSE %.5f' % curr_loss)
            
            cost.append(curr_loss)

    return cost


Creating a neural network with 1 feature. Then training that neural net.

In [ ]:
model = NN(X_train.size(1))
cost = train(model, X_train, y_train, n_epoch=50)


Plotting the graph relating the epoch and error.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(cost)), cost)

plt.ylabel('Mean Square Error')
plt.xlabel('Epoch')

plt.show()


Calculating the training and testing accuracies.

In [ ]:
ones = torch.ones(y_train.size())
zeros = torch.zeros(y_train.size())


In [ ]:
train_pred = model.forward(X_train)
train_acc = torch.mean((torch.where(train_pred > 0.5, ones, zeros).int() == y_train).float())

ones = torch.ones(y_test.size())
zeros = torch.zeros(y_test.size())

test_pred = model.forward(X_test)
test_acc = torch.mean((torch.where(test_pred > 0.5, ones, zeros).int() == y_test).float())

print('Training Accuracy: %.2f' % (train_acc * 100))
print('Testing Accuracy: %.2f' % (test_acc * 100))


---

# 3. Model Using Pytorch

In [ ]:
import torch
import sklearn
import torch.nn.functional as F


In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
# Create The dataset
X, y = make_classification(n_samples=1000, n_classes=2, n_features=4)

# Normalize the Data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Array to Tensor
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3,random_state=21)
print(X_train.shape)
print(X_test.shape)

In [ ]:
class NeuralNetworkPyTorch(nn.Module):
    def __init__(self, n_features):
        super(NeuralNetworkPyTorch, self).__init__()
        self.linear = nn.Linear(n_features, 1)

        self.linear.weight.detach().zero_()
        self.linear.bias.detach().zero_()

    def forward(self, x):
        out = self.linear(x)
        return out.view(-1)


Overloaded function `train()` to train the neural network built using PyTorch.

In [ ]:
def train(model, x, y, n_epoch=10, lr=0.001, seed=23, bsz=50):
    cost = []
    torch.manual_seed(seed)
    
    # Optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    for i in range(n_epoch):
        shuffledData = torch.randperm(y.size(0))
        batches = torch.split(shuffledData, bsz)

        for idx in batches:
            # 1. Compute the Output
            yHat = model.forward(x[idx])

            # 2. Compute the Loss
            loss = torch.nn.functional.mse_loss(yHat, y[idx])

            # 3. Compute the Gradients
            optimizer.zero_grad()
            loss.backward()

            # 4. Update the Model Parameters
            optimizer.step()

            # model.weight=model.weight-lr*grad_w
            # model.bias=model.bias-lr*grad_b

        # 5. Log and print the Loss
        with torch.no_grad():
            yHat = model.forward(x)
            curr_loss = loss_fn(yHat, y)
            print('Epoch: %3d ' % (i + 1), end="")
            print('| MSE % .5f' % curr_loss)
            cost.append(curr_loss)

    return cost


In [ ]:
n_feature = X_train.size(1)
model = NeuralNetworkPyTorch(n_features=n_feature)


In [ ]:
cost = train(model, X_train, y_train, n_epoch=50)


In [ ]:
plt.plot(range(len(cost)), cost)
plt.xlabel('No. of Epochs')
plt.ylabel('Mean Squared Error')
plt.plot()


In [ ]:
ones = torch.ones(y_train.size(0))
zero = torch.zeros(y_train.size(0))
train_pred = model.forward(X_train)

train_acc = torch.mean(
    (torch.where(train_pred > 0.5, ones, zero).int() == y_train).float())
# print(train_acc)

ones = torch.ones(y_test.size(0))
zero = torch.zeros(y_test.size(0))
test_pred = model.forward(X_test)

test_acc = torch.mean(
    (torch.where(test_pred > 0.5, ones, zero).int() == y_test).float())
print('Training Accuracy : %.2f' % train_acc)
print('Testing Accuracy: %.2f' % test_acc)


---

# 4. Logistic Regression

In [ ]:
def train(model, x, y, n_epoch=10, lr=0.001, seed=23, bsz=50):
    cost = []
    torch.manual_seed(seed)
    # Optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    for i in range(n_epoch):
        shuffledData = torch.randperm(y.size(0))
        batches = torch.split(shuffledData, bsz)

        for idx in batches:
            # 1. Compute the Output
            yHat = model.forward(x[idx])

            # 2. Compute the Loss
            loss = torch.nn.functional.binary_cross_entropy(
                torch.sigmoid(yHat), y[idx])

            # 3. Compute the Gradients
            optimizer.zero_grad()
            loss.backward()

            # 4. Update the Model Parameters
            optimizer.step()

            # model.weight=model.weight-lr*grad_w
            # model.bias=model.bias-lr*grad_b

        # 5. Log and print the Loss
        with torch.no_grad():
            yHat = model.forward(x)
            curr_loss = loss_fn(yHat, y)
            print('Epoch : %3d ' % (i + 1), end="")
            print('| MSE % .5f' % curr_loss)
            cost.append(curr_loss)

    return cost


In [ ]:
n_feature = X_train.size(1)
model = NeuralNetworkPyTorch(n_features=n_feature)


In [ ]:
cost = train(model, X_train, y_train, n_epoch=50)


In [ ]:
plt.plot(range(len(cost)), cost)
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.plot()


In [ ]:
ones = torch.ones(y_train.size(0))
zero = torch.zeros(y_train.size(0))
train_pred = model.forward(X_train)

train_acc = torch.mean(
    (torch.where(train_pred > 0.5, ones, zero).int() == y_train).float())
# print(train_acc)

ones = torch.ones(y_test.size(0))
zero = torch.zeros(y_test.size(0))
test_pred = model.forward(X_test)

test_acc = torch.mean(
    (torch.where(test_pred > 0.5, ones, zero).int() == y_test).float())
print('Training Accuracy : %.2f' % train_acc)
print('Testing Accuracy: %.2f' % test_acc)
